In [109]:
import pandas as pd
data = pd.read_csv("../0-Data/mushroom_dataset.csv")
pd.set_option('display.max_columns', len(data.columns))
print(len(data.columns))
data.head()

23


,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,mushroom
0,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,WHITE,TAPERING,BULBOUS,SMOOTH,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,PURPLE,SEVERAL,WOODS,EDIBLE
1,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,WHITE,TAPERING,BULBOUS,SMOOTH,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,BROWN,SEVERAL,WOODS,EDIBLE
2,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,PINK,TAPERING,BULBOUS,SMOOTH,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,PURPLE,SEVERAL,WOODS,EDIBLE
3,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,PINK,TAPERING,BULBOUS,SMOOTH,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,BROWN,SEVERAL,WOODS,EDIBLE
4,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,BROWN,TAPERING,BULBOUS,SMOOTH,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,PURPLE,SEVERAL,WOODS,EDIBLE


In [110]:
data.dtypes

cap-shape                   object
cap-surface                 object
cap-color                   object
bruises                     object
odor                        object
gill-attachment             object
gill-spacing                object
gill-size                   object
gill-color                  object
stalk-shape                 object
stalk-root                  object
stalk-surface-above-ring    object
stalk-surface-below-ring    object
stalk-color-above-ring      object
stalk-color-below-ring      object
veil-type                   object
veil-color                  object
ring-number                 object
ring-type                   object
spore-print-color           object
population                  object
habitat                     object
mushroom                    object
dtype: object

In [111]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize = (8,8)).set_facecolor("gray")
sns.heatmap(data.corr(method = "pearson", numeric_only=True))
plt.show()

ValueError: zero-size array to reduction operation fmin which has no identity

<Figure size 800x800 with 0 Axes>

In [ ]:
toEncode = [column for column in data.columns if column != 'mushroom']
for i in toEncode:
    encoded = (pd.get_dummies(data[i], dtype=float))
    data = pd.concat([data, encoded], axis=1)
    data.drop(i, axis=1, inplace=True)
pd.set_option("display.max_columns", len(data.columns))
print(len(data.columns))

118


In [ ]:
print(data['mushroom'].value_counts())
data['mushroom'] = data['mushroom'].map({'EDIBLE':0, 'POISONOUS':1})
print(data['mushroom'].value_counts())

mushroom
EDIBLE       4488
POISONOUS    3928
Name: count, dtype: int64
mushroom
0    4488
1    3928
Name: count, dtype: int64


In [ ]:
x = data.drop('mushroom', axis=1)
y = data['mushroom']

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
import numpy as np

normalizer = MinMaxScaler(feature_range=(0,1))
x_norm = normalizer.fit_transform(x)
pca = PCA(n_components=50)
x_pca = pca.fit_transform(x_norm)

variance = pca.explained_variance_ratio_
print("Explained variance:", variance)
print("Sum variance:", sum(variance))

param = {
'min_samples_split': np.array([4,5,6]),
'max_depth': np.array([1,2,3]),
'min_samples_leaf': np.array([4,5,6])
}

model = ExtraTreesClassifier(n_estimators=100, n_jobs=-1)
grid_model = GridSearchCV(estimator=model, param_grid=param, cv=5, n_jobs=-1)
grid_model.fit(x_pca, y)


print(f"Best samples split:", grid_model.best_estimator_.min_samples_split)
print(f"Best depth: {grid_model.best_estimator_.max_depth}")
print(f"Best samples leaf: {grid_model.best_estimator_.min_samples_leaf}")
print(f"R2: {grid_model.best_score_}")

Explained variance: [0.16301574 0.13562751 0.10468385 0.06811347 0.03925975 0.03667488
 0.03066417 0.02685855 0.02643941 0.02126487 0.02026812 0.01828312
 0.01732569 0.01684296 0.01527744 0.01505659 0.01477509 0.01401099
 0.01322323 0.01155915 0.01107787 0.0105335  0.01050667 0.00971112
 0.00876398 0.00861558 0.00812058 0.00758813 0.00732606 0.00685907
 0.00653906 0.00604131 0.0059928  0.00537338 0.00520702 0.00515751
 0.00459141 0.00456859 0.00443281 0.00434767 0.00416167 0.00408094
 0.0035743  0.00352107 0.00332674 0.00302109 0.0027926  0.00227548
 0.00219544 0.00205646]
Sum variance: 0.9815845116269658
Best samples split: 4
Best depth: 2
Best samples leaf: 6
R2: 0.894981109121112
